In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import torch

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

import utils.torchUtils as gnn

In [7]:
template = gnn.Dataset('data/MX_1200_MY_500-testing')

In [26]:
np.savetxt('data_csv/unscaled_node_shape.txt',np.array([ x.shape for x in unscaled_nodes]))
np.savetxt('data_csv/unscaled_node_x.txt',np.concatenate([x for x in unscaled_nodes]))
np.savetxt('data_csv/unscaled_edge_shape.txt',np.array([ x.shape for x in unscaled_edges]))
np.savetxt('data_csv/unscaled_edge_attr.txt',np.concatenate([x for x in unscaled_edges]))

In [8]:
model = gnn.GoldenGCN.load_from_checkpoint("models/golden_classifier/lightning_logs/version_3/checkpoints/epoch=9-step=39389.ckpt", dataset=template)
# model = gnn.GoldenGCN(dataset=dataset)

In [9]:
import utils.torchUtils.cpp_geometric as cpg 

cpg.export_model(model,template,'gnn_model_v3/')

Exporting model to: gnn_model_v3/
[model]
num_node_features = 5
node_feautres = m,pt,eta,phi,btag
num_edge_features = 1
edge_features = dr
num_layers = 7
layers = GCNConvMSG,GCNRelu,GCNConvMSG,GCNRelu,NodeLinear,EdgeLinear,GCNLogSoftmax
layer_shapes = 32,11,0,0,128,160,0,0,2,128,2,384,0,0

[scaler]
node_scale_min = 0.62060857,20.0,-2.4995117,-3.1415837,0.0010824203
node_scale_max = 432.17313,3893.0654,2.4995117,3.1415837,0.9995117
edge_scale_min = 0.0
edge_scale_max = 5.852164781833368


In [13]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in dataset[:1000]:
        node_o,edge_o = model(data)
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())

In [14]:
!mkdir -p data_csv

np.savetxt('data_csv/golden_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_node_x.txt',np.concatenate([x for x in node_x]))

In [15]:
np.savetxt('data_csv/golden_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [16]:
testing = [ data for data in dataset[:1000]]

In [17]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.conv1(data.x,data.edge_index,data.edge_attr)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_conv1_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_conv1_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_conv1_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_conv1_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [19]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.relu1(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_relu1_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_relu1_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_relu1_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_relu1_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [20]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.conv2(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_conv2_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_conv2_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_conv2_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_conv2_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [21]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.relu2(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_relu2_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_relu2_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_relu2_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_relu2_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [22]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.node_linear(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_node_linear_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_node_linear_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_node_linear_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_node_linear_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [23]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.edge_linear(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_edge_linear_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_edge_linear_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_edge_linear_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_edge_linear_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [24]:
with torch.no_grad():
    node_x, edge_attr = [],[]
    for data in testing:
        node_o,edge_o = model.log_softmax(data.node_o,data.edge_index,data.edge_o)
        data.node_o = node_o 
        data.edge_o = edge_o
        
        node_x.append(node_o.numpy())
        edge_attr.append(edge_o.numpy())
    
np.savetxt('data_csv/golden_log_softmax_node_shape.txt',np.array([ x.shape for x in node_x]))
np.savetxt('data_csv/golden_log_softmax_node_x.txt',np.concatenate([x for x in node_x]))
np.savetxt('data_csv/golden_log_softmax_edge_shape.txt',np.array([ x.shape for x in edge_attr]))
np.savetxt('data_csv/golden_log_softmax_edge_attr.txt',np.concatenate([x for x in edge_attr]))

In [33]:
model.conv1.weight.detach().numpy().shape

(32, 11)

In [42]:
output_matrix = lambda matrix : '\n'.join([ ' '.join('{:>12}'.format(f'{elem:0.6}') for elem in row) for row in matrix   ])

print(output_matrix(model.conv1.bias.detach().numpy()))

TypeError: 'numpy.float32' object is not iterable

In [44]:
model.conv1.bias.shape

torch.Size([32])